In [242]:
import pandas as pd

import numpy

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pandas as pd
import re
import numpy as np


In [47]:
df = pd.read_csv('./Full_Data_TI_and_Strategy_TOKAGGLE.csv') # This will delay ~2 minutes in load, please be patient

In [176]:
# Preprocess the data
data_copy = df.copy()
#df.drop(columns=['open_time'],inplace=True)

dates_close = [col for col in df.columns.tolist() if re.search('close_time',col)] 
df.drop(columns=dates_close,inplace=True)
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.fillna(method='ffill').fillna(method='bfill').dropna(axis=1)#['Fibonacci_Retracements_min']

In [49]:
# Define weights for different time frames
time_frame_weights = {'min': 1, '5min': 1.05, '10min':1.1, '15min':1.15, '30min': 1.30, 'hour': 2,'2hour': 2.2,'4hour': 3,'8hour': 3.8, '12hour': 4, '1day': 5, '1week': 6}

# Calculate the weighted strategies
for time_frame, weight in time_frame_weights.items():
    for column in df.columns:
        if time_frame in column and ('Strategy') in column and 'weighted' not in column:
            df[f'DS_weighted_{column}'] = df[column] * weight




/var/folders/fn/0t1gngls6x39txrdzml_kv3w0000gn/T/ipykernel_51325/3902602011.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'DS_weighted_{column}'] = df[column] * weight
/var/folders/fn/0t1gngls6x39txrdzml_kv3w0000gn/T/ipykernel_51325/3902602011.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'DS_weighted_{column}'] = df[column] * weight
/var/folders/fn/0t1gngls6x39txrdzml_kv3w0000gn/T/ipykernel_51325/3902602011.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

In [187]:
# Combine the weighted strategies
df['Combined_Strategy'] = df.filter(regex='weighted_Strategy').sum(axis=1)


In [86]:
positive_threshold = df['Combined_Strategy'].describe().T['75%'] + (df['Combined_Strategy'].describe().T['std'] * 1.5 )

# Normalize the combined strategies
positive_threshold = df['Combined_Strategy'].describe().T['75%'] + (df['Combined_Strategy'].describe().T['std'] * 1.5 ) # Set an appropriate psoitive threshold
negative_threshold = df['Combined_Strategy'].describe().T['25%'] - (df['Combined_Strategy'].describe().T['std'] * 1.5 )  # Set an appropriate negative threshold

df['Combined_Strategy_target'] = df['Combined_Strategy'].apply(
    lambda x: 1 if x > positive_threshold else (-1 if x < negative_threshold else 0)
)

In [186]:
#df.corr()['Combined_Strategy_target'] # 28 minutes just to get correlations!!

/var/folders/fn/0t1gngls6x39txrdzml_kv3w0000gn/T/ipykernel_51325/3936422342.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()['Combined_Strategy_target']


open_min                               0.168794
high_min                               0.168768
low_min                                0.168756
close_min                              0.168756
volume_min                            -0.037130
                                         ...   
DS_weighted_Strategy_CMF_12hour        0.194942
DS_weighted_Strategy_CCI_20_12hour    -0.119762
DS_weighted_Strategy_OBV_roc_12hour    0.281193
Combined_Strategy                      0.428305
Combined_Strategy_target               1.000000
Name: Combined_Strategy_target, Length: 924, dtype: float64

In [200]:
X.shape , y.shape , X.columns , y

((1440, 923),
 (1440,),
 Index(['open_min', 'high_min', 'low_min', 'close_min', 'volume_min',
        'quote_volume_min', 'count_min', 'taker_buy_volume_min',
        'taker_buy_quote_volume_min', 'ignore_min',
        ...
        'DS_weighted_Strategy_Stochastic_Oscillator_12hour',
        'DS_weighted_Strategy_ROC_12_12hour',
        'DS_weighted_Strategy_RVI_14_12hour',
        'DS_weighted_Strategy_Bollinger_Bands_12hour',
        'DS_weighted_Strategy_Donchian_Channels_12hour',
        'DS_weighted_Strategy_MFI_14_12hour', 'DS_weighted_Strategy_CMF_12hour',
        'DS_weighted_Strategy_CCI_20_12hour',
        'DS_weighted_Strategy_OBV_roc_12hour', 'Combined_Strategy'],
       dtype='object', length=923),
 1168860    0
 1168861    0
 1168862    0
 1168863    0
 1168864    0
           ..
 1170295    0
 1170296    0
 1170297    0
 1170298    0
 1170299    0
 Name: Combined_Strategy_target, Length: 1440, dtype: int64)

In [243]:
'''import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#

# Correlation between different variables
#
corr = df.corr()
#
# Set up the matplotlib plot configuration
#
f, ax = plt.subplots(figsize=(12, 10))
#
# Generate a mask for upper traingle
#
mask = np.triu(np.ones_like(corr, dtype=bool))
#
# Configure a custom diverging colormap
#
cmap = sns.diverging_palette(230, 20, as_cmap=True)
#
# Draw the heatmap
#
sns.heatmap(corr, annot=True, mask = mask, cmap=cmap)'''

'import pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport seaborn as sns\n#\n\n# Correlation between different variables\n#\ncorr = df.corr()\n#\n# Set up the matplotlib plot configuration\n#\nf, ax = plt.subplots(figsize=(12, 10))\n#\n# Generate a mask for upper traingle\n#\nmask = np.triu(np.ones_like(corr, dtype=bool))\n#\n# Configure a custom diverging colormap\n#\ncmap = sns.diverging_palette(230, 20, as_cmap=True)\n#\n# Draw the heatmap\n#\nsns.heatmap(corr, annot=True, mask = mask, cmap=cmap)'

In [245]:
df = df.iloc[-150000:]

In [251]:
df.

,open_time,open_min,high_min,low_min,close_min,volume_min,quote_volume_min,count_min,taker_buy_volume_min,taker_buy_quote_volume_min,...,DS_weighted_Strategy_ROC_12_12hour,DS_weighted_Strategy_RVI_14_12hour,DS_weighted_Strategy_Bollinger_Bands_12hour,DS_weighted_Strategy_Donchian_Channels_12hour,DS_weighted_Strategy_MFI_14_12hour,DS_weighted_Strategy_CMF_12hour,DS_weighted_Strategy_CCI_20_12hour,DS_weighted_Strategy_OBV_roc_12hour,Combined_Strategy,Combined_Strategy_target
1020300,2022-12-21 20:00:00,16748.4,16748.4,16738.3,16738.3,60.260,1.008973e+06,414,6.400,107153.1542,...,-4,4,0,0,-4,4,0,-4,-8.2,0
1020301,2022-12-21 20:01:00,16738.4,16741.8,16730.0,16741.8,63.982,1.070851e+06,535,44.837,750440.0837,...,-4,4,0,0,-4,4,0,-4,-9.2,0
1020302,2022-12-21 20:02:00,16741.8,16746.5,16741.8,16746.5,13.107,2.194701e+05,230,10.391,173997.1500,...,-4,4,0,0,-4,4,0,-4,-11.2,0
1020303,2022-12-21 20:03:00,16746.5,16749.6,16735.7,16738.9,69.113,1.157211e+06,463,37.959,635542.8611,...,-4,4,0,0,-4,4,0,-4,-10.2,0
1020304,2022-12-21 20:04:00,16738.8,16744.5,16727.1,16727.1,53.486,8.952788e+05,487,17.388,290993.9791,...,-4,4,0,0,-4,4,0,-4,-8.2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170295,2023-04-04 23:55:00,28166.1,28194.0,28166.0,28169.0,64.686,1.823072e+06,848,27.317,769848.4404,...,-4,4,0,0,-4,4,0,-4,-24.8,0
1170296,2023-04-04 23:56:00,28168.9,28168.9,28168.8,28168.8,5.743,1.617736e+05,60,2.051,57774.4139,...,-4,4,0,0,-4,4,0,-4,-23.8,0
1170297,2023-04-04 23:57:00,28168.6,28168.6,28159.6,28159.6,10.936,3.079946e+05,155,0.301,8476.1866,...,-4,4,0,0,-4,4,0,-4,-23.8,0
1170298,2023-04-04 23:58:00,28159.6,28159.6,28155.3,28158.9,19.541,5.502004e+05,168,13.962,393113.1367,...,-4,4,0,0,-4,4,0,-4,-23.8,0


In [215]:
df.corr()

/var/folders/fn/0t1gngls6x39txrdzml_kv3w0000gn/T/ipykernel_51325/1134722465.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,open_min,high_min,low_min,close_min,volume_min,quote_volume_min,count_min,taker_buy_volume_min,taker_buy_quote_volume_min,ignore_min,...,DS_weighted_Strategy_ROC_12_12hour,DS_weighted_Strategy_RVI_14_12hour,DS_weighted_Strategy_Bollinger_Bands_12hour,DS_weighted_Strategy_Donchian_Channels_12hour,DS_weighted_Strategy_MFI_14_12hour,DS_weighted_Strategy_CMF_12hour,DS_weighted_Strategy_CCI_20_12hour,DS_weighted_Strategy_OBV_roc_12hour,Combined_Strategy,Combined_Strategy_target
open_min,1.000000,0.999566,0.999567,0.999339,-0.035615,-0.025278,-0.050425,-0.034571,-0.024707,NaN,...,0.678871,NaN,-0.453736,NaN,NaN,NaN,-0.783436,NaN,0.091100,NaN
high_min,0.999566,1.000000,0.999236,0.999650,-0.018085,-0.007683,-0.032232,-0.014430,-0.004528,NaN,...,0.678976,NaN,-0.454268,NaN,NaN,NaN,-0.783837,NaN,0.088514,NaN
low_min,0.999567,0.999236,1.000000,0.999625,-0.051905,-0.041495,-0.068018,-0.047182,-0.037236,NaN,...,0.678497,NaN,-0.452845,NaN,NaN,NaN,-0.783192,NaN,0.090803,NaN
close_min,0.999339,0.999650,0.999625,1.000000,-0.034262,-0.023850,-0.049460,-0.027833,-0.017913,NaN,...,0.679044,NaN,-0.453809,NaN,NaN,NaN,-0.783769,NaN,0.089349,NaN
volume_min,-0.035615,-0.018085,-0.051905,-0.034262,1.000000,0.999725,0.969329,0.974307,0.973174,NaN,...,-0.020147,NaN,-0.008876,NaN,NaN,NaN,0.027689,NaN,-0.062606,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DS_weighted_Strategy_CMF_12hour,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DS_weighted_Strategy_CCI_20_12hour,-0.783436,-0.783837,-0.783192,-0.783769,0.027689,0.018978,0.038589,0.022123,0.013807,NaN,...,-0.589017,NaN,0.407369,NaN,NaN,NaN,1.000000,NaN,0.016019,NaN
DS_weighted_Strategy_OBV_roc_12hour,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Combined_Strategy,0.091100,0.088514,0.090803,0.089349,-0.062606,-0.062866,-0.068343,-0.069693,-0.069876,NaN,...,0.266454,NaN,0.040282,NaN,NaN,NaN,0.016019,NaN,1.000000,NaN


In [249]:
import pandas as pd
import seaborn as sns
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

'''# Compute the correlation coefficients
correlations = df.corr()['Combined_Strategy_target'].drop('Combined_Strategy_target')

# Visualize the correlations using a heatmap
sns.heatmap(correlations.to_frame(), annot=True, cmap='coolwarm')
'''

# Perform Recursive Feature Elimination (RFE) using RandomForestClassifier
model = RandomForestClassifier()
rfe = RFE(estimator=model, n_features_to_select=10)
X = df.drop(['open_time','Combined_Strategy_target'], axis=1)
y = df['Combined_Strategy_target']
rfe.fit(X, y)

# Get the selected features
selected_features_rfe = X.columns[rfe.support_]

# Get feature importance from the same tree-based model
feature_importances = rfe.estimator_.feature_importances_

# Sort features by importance and select the top-N features
sorted_indices = feature_importances.argsort()[::-1]
top_n_features = 10
selected_features_tree = X.columns[sorted_indices[:top_n_features]]

# Print the selected features
print("Selected features using RFE:", selected_features_rfe)
print("Selected features using tree-based model:", selected_features_tree)


KeyboardInterrupt: 

In [237]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Use the selected features from the previous steps
selected_features = selected_features_rfe  # or selected_features_tree

# Split the data into training and validation sets
X_train, X_mid, y_train, y_mid = train_test_split(X[selected_features], y, test_size=0.3, random_state=42)

# Split the data into training and validation sets
X_val, X_test, y_val, y_test = train_test_split(X_mid[selected_features], y_mid, test_size=0.5, random_state=42)

# Train a simple logistic regression model
baseline_model = RandomForestClassifier()
baseline_model.fit(X_train, y_train)


RandomForestClassifier()

In [238]:
y_val.value_counts()

0    2250
Name: Combined_Strategy_target, dtype: int64

In [239]:
# Make predictions on the validation set
y_pred = baseline_model.predict(X_val)

# Calculate evaluation metrics
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred, average='weighted')
recall = recall_score(y_val, y_pred, average='weighted')
f1 = f1_score(y_val, y_pred, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")


Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-score: 1.0


In [234]:
import xgboost as xgb
from skopt import BayesSearchCV
from skopt.space import Real, Integer

# Create an XGBoost classifier
complex_model = xgb.XGBClassifier()

search_space = {
    'learning_rate': (0.01, 0.5),
    'max_depth': (3, 10),
    'n_estimators': (50, 500),
    'min_child_weight': (1, 10),
    'subsample': (0.5, 1),
    'gamma': (0, 1),
    'colsample_bytree': (0.1, 1),
}



# Define the hyperparameter search space
opt_space = {
    'learning_rate': Real(0.01, 0.5),
    'max_depth': Integer(3, 10),
    'n_estimators': Integer(50, 500),
    'min_child_weight': Integer(1, 10),
    'subsample': Real(0.5, 1),
    'gamma': Real(0, 1),
    'colsample_bytree': Real(0.1, 1),
}

# Create a BayesSearchCV object
bayes_opt = BayesSearchCV(
    complex_model,
    opt_space,
    n_iter=50,
    cv=3,
    n_jobs=-1,
    scoring='accuracy',
    random_state=42
)

# Fit the Bayesian optimization model
bayes_opt.fit(X_train, y_train)

# Get the best hyperparameters found by Bayesian optimization
best_params = bayes_opt.best_params_
print("Best hyperparameters found:", best_params)

# Train the complex model with the best hyperparameters on the full training set
best_complex_model = xgb.XGBClassifier(**best_params)
best_complex_model.fit(X_train, y_train)

# Make predictions on the validation set
y_pred_complex = best_complex_model.predict(X_val)

# Calculate evaluation metrics for the complex model
accuracy_complex = accuracy_score(y_val, y_pred_complex)
precision_complex = precision_score(y_val, y_pred_complex, average='weighted')
recall_complex = recall_score(y_val, y_pred_complex, average='weighted')
f1_complex = f1_score(y_val, y_pred_complex, average='weighted')

print(f"Complex model accuracy: {accuracy_complex}")
print(f"Complex model precision: {precision_complex}")
print(f"Complex model recall: {recall_complex}")
print(f"Complex model F1-score: {f1_complex}")




Best hyperparameters found: OrderedDict([('colsample_bytree', 0.46909356296798244), ('gamma', 0.7277257431773251), ('learning_rate', 0.4671053194354386), ('max_depth', 5), ('min_child_weight', 7), ('n_estimators', 236), ('subsample', 0.675465667449572)])
Complex model accuracy: 1.0
Complex model precision: 1.0
Complex model recall: 1.0
Complex model F1-score: 1.0


In [235]:
print("Baseline model performance:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")

print("\nComplex model performance:")
print(f"Accuracy: {accuracy_complex}")
print(f"Precision: {precision_complex}")
print(f"Recall: {recall_complex}")
print(f"F1-score: {f1_complex}")


Baseline model performance:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-score: 1.0

Complex model performance:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-score: 1.0


In [241]:
best_complex_model.predict(X_test)

array([0, 0, 0, ..., 0, 0, 0])

In [250]:

# Make predictions on the validation set
y_pred_complex = best_complex_model.predict(X_test)

# Calculate evaluation metrics for the complex model
accuracy_complex = accuracy_score(y_val, y_pred_complex)
precision_complex = precision_score(y_val, y_pred_complex, average='weighted')
recall_complex = recall_score(y_val, y_pred_complex, average='weighted')
f1_complex = f1_score(y_val, y_pred_complex, average='weighted')

print(f"Test model accuracy: {accuracy_complex}")
print(f"Test model precision: {precision_complex}")
print(f"Test model recall: {recall_complex}")
print(f"Test model F1-score: {f1_complex}")

Test model accuracy: 1.0
Test model precision: 1.0
Test model recall: 1.0
Test model F1-score: 1.0
